# Pandas 5

## Chapter 8

### Combining and Merging Datasets
- Database-Style DataFrame Joins
- Merging on Index
- Concatenating Along an Axis
- Combining Data with Overlap

### Database-Style DataFrame Joins

In [2]:
import pandas as pd
import numpy as np

In [4]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [5]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd','b'],'data2': range(4)})
df2

,key,data2
0,a,0
1,b,1
2,d,2
3,b,3


In [6]:
# Note that I didn’t specify which column to join on. If that information 
# is not specified, merge uses the overlapping column names as the keys.

# many to one join
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,6,1
5,b,6,3
6,a,2,0
7,a,4,0
8,a,5,0


In [7]:
pd.merge(df2, df1)

,key,data2,data1
0,a,0,2
1,a,0,4
2,a,0,5
3,b,1,0
4,b,1,1
5,b,1,6
6,b,3,0
7,b,3,1
8,b,3,6


In [8]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,6,1
5,b,6,3
6,a,2,0
7,a,4,0
8,a,5,0


In [4]:
# If the column names are different in each object, you can specify them separately:

df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [5]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [11]:
# outer union , inner# intesection
pd.merge(df3, df4, left_on='lkey', right_on='rkey',how='outer')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [12]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [13]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [14]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [16]:
pd.merge(df1, df2, on="key", how="right")

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,b,0.0,1
3,b,1.0,1
4,b,5.0,1
5,a,2.0,2
6,a,4.0,2
7,b,0.0,3
8,b,1.0,3
9,b,5.0,3


- To merge with multiple keys, pass a list of column names
- To determine which key combinations will appear in the result depending on the choice of merge method, think of the multiple keys as forming an array of tuples to be used as a single join key (even though it’s not actually implemented that way)

In [18]:
left = pd.DataFrame({'userId': ['foo', 'foo', 'bar'],'userName': ['one', 'two', 'one'],'lval': [1, 2, 3]})
left

,userId,userName,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [19]:
right = pd.DataFrame({'userId': ['foo', 'foo', 'bar', 'bar'],'userName': ['one', 'one', 'one', 'two'],'rval': [4, 5, 6, 7]})
right

,userId,userName,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [20]:
pd.merge(left, right, on=['userId','userName'], how='right')

,userId,userName,lval,rval
0,foo,one,1.0,4
1,foo,one,1.0,5
2,bar,one,3.0,6
3,bar,two,NaN,7


In [21]:
pd.merge(left, right, on=['userId','userName'],how='left')

,userId,userName,lval,rval
0,foo,one,1,4.0
1,foo,one,1,5.0
2,foo,two,2,NaN
3,bar,one,3,6.0


In [22]:
pd.merge(right,left, on=['userId','userName'],how='right')

,userId,userName,rval,lval
0,foo,one,4.0,1
1,foo,one,5.0,1
2,foo,two,NaN,2
3,bar,one,6.0,3


In [23]:
pd.merge(right,left, on=['userId','userName'],how='left')

,userId,userName,rval,lval
0,foo,one,4,1.0
1,foo,one,5,1.0
2,bar,one,6,3.0
3,bar,two,7,NaN


## Merging on Index
In some cases, the merge key(s) in a DataFrame will be found in its index. In this
case, you can pass left_index=True or right_index=True (or both) to indicate that
the index should be used as the merge `

In [25]:
left1 = pd.DataFrame({'key': ['c', 'd', 'a', 'a', 'b', 'c'],'value': range(6)})
left1

,key,value
0,c,0
1,d,1
2,a,2
3,a,3
4,b,4
5,c,5


In [26]:
right1 = pd.DataFrame({'group_val': [3.5, 7]})
right1

,group_val
0,3.5
1,7.0


In [29]:
pd.merge(left1, right1, left_index=True, right_index=True) # how='outer'

,key,value,group_val
0,c,0,3.5
1,d,1,7.0


In [30]:
pd.merge(left1, right1, left_index=True, right_index=True, how='outer')

,key,value,group_val
0,c,0,3.5
1,d,1,7.0
2,a,2,NaN
3,a,3,NaN
4,b,4,NaN
5,c,5,NaN


DataFrame has a convenient join instance for merging by index. It can also be used
to combine together many DataFrame objects having the same or similar indexes but
non-overlapping columns.

### Combining and Merging Datasets
Data contained in pandas objects can be combined together in a number of ways:
- **pandas.merge** connects rows in DataFrames based on one or more keys. This
will be familiar to users of SQL or other relational databases, as it implements
database join operations.

- **pandas.concat** concatenates or “stacks” together objects along an axis.

- The **combine_first** instance method enables splicing together overlapping data
to fill in missing values in one object with values from another.

In [12]:
import requests

In [13]:
url = 'https://jsonplaceholder.typicode.com/todos/'

In [14]:
resp = requests.get(url)

ConnectionError: HTTPSConnectionPool(host='jsonplaceholder.typicode.com', port=443): Max retries exceeded with url: /todos/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fda603d40d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [ ]:
resp

In [15]:
import sqlite3

In [16]:
query = "CREATE TABLE test (id INTEGER, username VARCHAR(20));"

In [17]:
conn = sqlite3.connect('db_data.sqlite')

In [18]:
conn

In [21]:
try: conn.execute(query)
except Exception as e: print(e)

table test already exists


In [22]:
conn.commit()

In [23]:
data = [(1, 'Jonathan'),(2, 'Saqib'),(3, 'Umair'), (222,2344)]
data

[(1, 'Jonathan'), (2, 'Saqib'), (3, 'Umair'), (222, 2344)]

In [24]:
stmt = "INSERT INTO test VALUES (?, ?)"

In [25]:
conn.executemany(stmt, (data))

In [27]:
conn.commit()

In [34]:
cursor = conn.execute("SELECT * FROM test")

In [29]:
cursor

In [30]:
for x in cursor:
    print(x)

(1, 'Jonathan')
(2, 'Saqib')
(3, 'Umair')
(222, '2344')


In [36]:
cursor = conn.execute("SELECT * FROM test")

In [37]:
rows = cursor.fetchall()
rows

[(1, 'Jonathan'), (2, 'Saqib'), (3, 'Umair'), (222, '2344')]

In [42]:
conn.close()

In [41]:
pd.DataFrame(rows,  columns=['id', 'username'])

,id,username
0,1,Jonathan
1,2,Saqib
2,3,Umair
3,222,2344


In [44]:
with sqlite3.connect('db_data.sqlite') as con:
    cursor = con.execute('SELECT * FROM test')
    for x in cursor:
        print(x)

(1, 'Jonathan')
(2, 'Saqib')
(3, 'Umair')
(222, '2344')
